<a href="https://colab.research.google.com/github/Massittha/Data-portfolio/blob/main/clean_covid_data_viz.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Data cleaning for COVID-19 Report on Looker Studio

Data source: https://github.com/owid/covid-19-data/tree/master/public/data

Import pandas and read the csv file

In [26]:
import pandas as pd
df = pd.read_csv('owid-covid-data.csv')


In [27]:
# check no. of rows and columns
df.shape

(350049, 67)

See the information of the dataset

In [28]:
# see dataset info
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 350049 entries, 0 to 350048
Data columns (total 67 columns):
 #   Column                                      Non-Null Count   Dtype  
---  ------                                      --------------   -----  
 0   iso_code                                    350049 non-null  object 
 1   continent                                   333398 non-null  object 
 2   location                                    350049 non-null  object 
 3   date                                        350049 non-null  object 
 4   total_cases                                 312088 non-null  float64
 5   new_cases                                   340457 non-null  float64
 6   new_cases_smoothed                          339198 non-null  float64
 7   total_deaths                                290501 non-null  float64
 8   new_deaths                                  340511 non-null  float64
 9   new_deaths_smoothed                         339281 non-null  float64
 

## Data preparation

In [29]:
# select only interested columns
columns = ['date','iso_code','continent','location','new_cases','new_deaths','icu_patients','hosp_patients',\
            'new_tests','people_vaccinated','people_fully_vaccinated','population']
covid_viz = df[columns]

covid_viz.shape

(350049, 12)

In [30]:
# some data cleaning

# because rows with "OWID" as the 'iso_code' contain continent wise records
# which overlap or duplicate country wise records.

# As I would like the report in Looker studio to aggregate country wise records,
# I removed rows containing "OWID"

remove = covid_viz[covid_viz['iso_code'].str.contains("OWID")].index
covid_viz = covid_viz.drop(remove)

# check number of rows left
covid_viz.shape


(326200, 12)

In [31]:
# check null values
covid_viz.isna().sum()

date                            0
iso_code                        0
continent                       0
location                        0
new_cases                    3748
new_deaths                   3694
icu_patients               292879
hosp_patients              292400
new_tests                  250900
people_vaccinated          265818
people_fully_vaccinated    269174
population                      0
dtype: int64

In [35]:
# fill na with 0
#'hosp_patients','icu_patients'

covid_viz[['new_cases','new_deaths','new_tests']] = \
covid_viz[['new_cases','new_deaths','new_tests']].fillna(value = 0)

In [21]:
# create a list of all countries in the dataset
countries = list(covid_viz.location.unique())

In [36]:
# fill forward 'people_vaccinated' and 'people_fully_vaccinated' in each country
for country in countries:
    covid_viz.loc[covid_viz['location'] == country,['people_vaccinated','people_fully_vaccinated']] = covid_viz[['people_vaccinated','people_fully_vaccinated']][covid_viz['location'] == country].fillna(method='ffill')



In [37]:
# fill the rest with 0
covid_viz[['people_vaccinated','people_fully_vaccinated']] = \
covid_viz[['people_vaccinated','people_fully_vaccinated']].fillna(value = 0)

In [38]:
# check null values again
# the null values remained at this point are intended
# to be shown as 'no data' in the report/dashboard

covid_viz.isna().sum()

date                            0
iso_code                        0
continent                       0
location                        0
new_cases                       0
new_deaths                      0
icu_patients               292879
hosp_patients              292400
new_tests                       0
people_vaccinated               0
people_fully_vaccinated         0
population                      0
dtype: int64

## Export data as a csv file

In [39]:
# create a csv file for Looker Studio
covid_viz.to_csv('covid_data_viz.csv',index = False)